# Some fun with functions and fractals (Informatics II)

author: Tsjerk Wassenaar

The topic of this tutorial is advanced functions in Python. This consists of several aspects:

* Functions with variable arguments lists (\*args and \*\*kwargs)
* Recursive functions
* Functions as objects
* Functions returning functions (closures)

The last two of these are mainly to give a bit of feel of what functions are (in Python) and what you can do with them and are there for *passive learning*. The first two are part of the core of Informatics 2. 

The aspects of functions named above are here demonstrated by making fractals, which are mathematical images with *scaled symmetry*: the image consists of smaller copies of itself, which consist of smaller copies of themselves. Such fractals actually occur in biological systems, and can be seen in the structures of weeds and trees. Nice examples are to be find <a href="http://paulbourke.net/fractals/fracintro/">here</a>

We'll be drawing the fractals first in 2D with turtle graphics. Towards the end, we'll be able to extend to 3D and generate a fractal structure for drawing with **pypovray** (optional).

Just to set a few things straight:

* You **don't** need to know fractals, L-systems and any of the specific ones named and used.
* You **do** need to understand *recursive functions* and *recursion depth*


* You **don't** need to know (reproduce) the functions used in this tutorial
* You **do** need to understand how the functions work and be able to put them to use in the template


* You **do** need to write a template and put the functions in to make this work. Although... you can also work interactively to try things out.

When writing a turtle program using the template, you can start with the following basic main function, to keep the image until you press the _any_ key:

In [ ]:
def main(args):
    """Docstring goes here"""
    
    # Preparation
    
    # Processing

    # Finishing
    input('Press any key to continue')

    return 0

## Recursion

In [ ]:
def recurse():
    print("A recursive function is a function that calls itself, like:")
    recurse()
    
recurse()

So, a recursive function is one that calls itself.

Well, that's that. So, we can continue to the next topic...

On the other hand, maybe it is good to think about how that works in practice and good to think about why you'd want to do that. Just to set things straight: there is nothing you can do with recursion that you can't do with a for loop and creative use of (nested) lists. However, in some cases, you'll have to get really creative, and you may be better off if you can split your problem into parts and do try the same function/strategy on the parts.

A classic example of a recursive function is the factorial. The factorial of an integer is the joint product of that number and *all* foregoing positive numbers. It's typically written as n!. So the outcome of 5! is 5\*4\*3\*2\*1 = 120. There is one additional rule: by definition 0! = 1

With that, we can write a factorial function. First, look at the for-loop way:

In [ ]:
def factorial(n):
    result = 1
    for num in range(n,0,-1):
        result *= num
    return result

We can turn this into a recursive function, by considering two cases: n is 0, or n is not. If n is 0, then the result is 1 (by definition). If n is not (yet) zero, then the result is n * (n-1)!, so n times the factorial of n-1:

In [ ]:
def factorial(n):
    if not n:
        return 1
    return n * factorial(n - 1)

### Assignment:

Write a program factorial.py that takes a number as command line argument and prints the factorial of that number. Start with a correct template and use the recursive function. Write docstrings!

## Fractals and recursion

For fractals, we'll focus on Lindenmayer fractals (L-systems). These are written as a series of steps, like forward, right, and left. The trick is that a step can be replaced by a sequence of steps, in which steps can be replaced by that sequence of steps again, and so on and so forth. Because of time, that has to end somewhere, and we'll call that the depth of the sequence.

So, the L-system consists of:

* the **axiom**: the start sequence
* the **rules**: the replacement rules
* the **depth**: the depth of the recursion

The result is a sequence of instructions (forward, right, and left) that we can nicely pass to our turtle friend Don.

In [ ]:
import turtle

don = turtle.Turtle(shape="turtle")

We start off with the **Hilbert function**, which can be written as an L-system (thanks Wikipedia):


* axiom: A
* rules:
 - A → -BF+AFA+FB-
 - B → +AF-BFB-FA+

Here, "F" means "draw forward", "−" means "turn left 90°", "+" means "turn right 90°" (see turtle graphics), and "A" and "B" are ignored during drawing.

**This means** that we start with 'A', and then replace 'A' with '-BF+AFA+FB-'. In the result, we replace each 'A' with that same string, but each B is replaced with '+AF-BFB-FA+'. And we can repeat that...

In [ ]:
def hilbert(depth, sequence='A'):
    if not depth: 
        return sequence
    out = []
    for character in sequence:
        if character == 'A':
            out.extend('-BF+AFA+FB-')
        elif character == 'B':
            out.extend('+AF-BFB-FA+')
        else:
            out.append(character)
    return hilbert(depth - 1, out)

In [ ]:
print("".join(hilbert(0)))

In [ ]:
print("".join(hilbert(1)))

In [ ]:
print("".join(hilbert(2)))

In [ ]:
print("".join(hilbert(3)))

Now, for each F in the sequence don goes forward, for each - he goes left and for each + he goes right. We can write this with an if/elif clause:

In [ ]:
for char in hilbert(3):
    if char == 'F':
        don.forward(10)
    elif char == '+':
        don.right(90)
    elif char == '-':
        don.left(90)


This is a function specific for the Hilbert function, which is pretty cool, as it generates a maze-like drawing. But there are many other interesting L-systems, and we can capture more of them, using the advanced function syntax, which allows us to specify an arbitrary number of keyword arguments:

In [ ]:
def l_system(depth, axiom, **rules):
    if not depth:
        return axiom
    
    # Basic, most straight-forward implementation
    # Note 1: it doesn't matter if axiom is a string or a list
    # Note 2: consider the difference between .extend() and .append()
    out = []
    for char in axiom:
        if char in rules:
            out.extend(rules[char])
        else:
            out.append(char)
    
    # Two alternative implementations. If you want to try 
    # an alternative, comment out the original first.
    # It won't change the answer, but it will take more time
    # if you keep the code active.
    
    # I. Alternative implementation using dict.get
    # --------------------------------------------
    # out = []
    # for char in axiom:
    #     out.extend(rules.get(char, [char]))
    
    # II. Alternative implementation in one line using list comprehension
    # -------------------------------------------------------------------
    # out = [i for char in axiom for i in rules.get(char, char)]
    
    # Note 3: See how comments are used to annotate the code... :)
    
    return l_system(depth - 1, out, **rules)

With this, we can write the Hilbert function much shorter:

In [ ]:
def hilbert(depth):
    return l_system(depth, axiom='A', A='-BF+AFA+FB-', B='+AF-BFB-FA+')

And we can write a Sierpinski gasket, using

* **axiom**: f
* **rules**:
  - F: f+F+f
  - f: F-f-F

With the note that both f and F mean forward.

In [ ]:
def sierpinski_gasket(depth):
    return l_system(depth, axiom='f', F='f+F+f', f='F-f-F')

In [ ]:
for char in sierpinski_gasket(7):
    if char in 'Ff':
        don.forward(1)
    elif char == '+':
        don.right(60)
    elif char == '-':
        don.left(60)

The next step is getting rid of the if/elif/elif/... clause, to make the handling of actions a bit nicer.

## Functions as objects

Getting rid of an if/elif/.. construct typically involves introducing a dictionary. A good reason to do that is that a dictionary requires less bookkeeping. However, in our case, we deal with actions, not values. Then again, actions are processes, which can be described as functions. So, we'll put *functions* in a dictionary!

Again, what we'll do is just a different way, whether it's actually better depends on the situation.

The idea of putting functions in a dictionary hinges on using functions as objects. Functions are objects that are *callable*: you can add parentheses to invoke the action. Without parentheses, it's just the function object. Consider the following example:

In [ ]:
blabla = print
blabla("Hello World!")

So, we assign the print *object* to a new variable, called *blabla*, and we can use that name too as print function. Likewise, we can store the function in a tuple, list, set or dictionary:

In [ ]:
actions = {"p": print}
actions["p"]("Hello World!")

Notice what happens there. We store the print function in a dictionary, bound to the key "p". Then we use the key "p" to get the corresponding value from the dictionary, and we *call* the process, by adding the parentheses with the argument "Hello World!".

Now let's do that with the actions for Don.

In [ ]:
def forward(turt, step=5):
    turt.forward(step)


def right(turt, angle=90):
    turt.right(angle)

    
def left(turt, angle=90):
    turt.left(angle)


actions = {'F': forward, 'f': forward, '+': right, '-': left}

Take a moment and think about the function definitions (and write docstrings!). The functions are very simple, but it's not easily possible to actually put the turtle functions in the dictionary. Well, actually it *is* easy once you know how, but it's not actually easy to call them nicely then. The approach above is easier to deal with:

In [ ]:
for char in hilbert(5):
    if char in actions:
        actions[char](don)

## Functions with function definitions

And now we'll be taking a step further. Note: **this is not mandatory stuff for the exam**. However, the following things may give you a good feel for the idea of functions being objects, just like (other) variables. So, to allow the actions to have different angles/steps, to deal with the Sierpinski thing, we'll generate the actions dictionary with a function, in which we can set the angle and the step:

In [ ]:
def actions(step, angle):
    
    def forward(turt):
        turt.forward(step)
        
    def right(turt):
        turt.right(angle)
        
    def left(turt):
        turt.left(angle)
    
    return {'F': forward, 'f': forward, '+': right, '-': left}

Now, put this function in your code and write the docstring. Take a moment to see what is happening here. *Within the function **actions** we define three functions, which take a **turt** as argument. The three functions are put in a dictionary, and this dictionary is returned.* The dictionary can then be used:

In [ ]:
actions_dict = actions(step=1, angle=60)
for char in sierpinski_gasket(7):
    if char in actions_dict:
        actions_dict[char](don)

## Solutions

Solutions for the excercises are given  below. Programming is like playing the piano: excercise, excercise, excercise. You learn most from typing each single word yourself. If you have no clue what to do you can have a look, but only after your first and second try. Remember there are many ways to come to a solution. Your idea might be valid as well. Discuss with your teacher the outcome or the differences.

<p><a href="sierpinski.py">sierpinski.py</a></p>